This notebook is associated with the paper "The relative class number one problem for function fields, I" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5rc4) and depends on Magma (tested using version
2.26-9); it also requires the pandas and openpyxl libraries (`sage --pip install openpyxl/pandas`).

In this notebook, we compute cyclic purely geometric extensions with $q=2$ using the upper bound on Weil polynomials. This depends on the exhaustion over Weil polynomials performed in the notebook "Weil polynomial bound for q=2" in this repository, the tabulation of curves of genus $\leq 4$ in LMFDB (including data by Xarles in genus 4), and the data file "genus5byweilpoly.txt" created by the notebook "Genus 5 curves.ipynb" based on the data of Dragutinović in genus 5. Allow 1 hour for completion.

In [1]:
load("../Shared/weil_poly_utils.sage")
load("../Shared/cyclic_covers.sage")

# Setup

Read data about the exhaustion from an Excel spreadsheet. Note that we only use the data about base curves $C$, not covering curves $C'$ (except at the end to match up the results).

In [2]:
import pandas, itertools, re
from collections import defaultdict

In [3]:
df = pandas.read_excel('../Shared/polys.xlsx', index_col=int(0))

In [4]:
if "Unnamed: 0" in df:
    del df["Unnamed: 0"]
if "Cyclic" in df:
    del df["Cyclic"]
print(list(df))

['d', 'g', "g'", 'Label of J(C)', '#J(C)(F_2)', '#J(C)(F_4)', 'Counts of C', "Counts of C'"]


In [5]:
candidates = {}
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    if (d,g,g1) not in candidates:
        candidates[d,g,g1] = []
    s = r["Label of J(C)"]
    if s not in candidates[d,g,g1]:
        candidates[d,g,g1].append(s)

# Defining some curves

Table of curves with a given Weil polynomial, taken from LMFDB.

In [6]:
curves_by_poly = {}

In [7]:
P1.<x> = GF(2)[]
curves_by_poly[1] = {
    '1.2.ac': [(P1(1), x^3+x+1)],
    '1.2.ab': [(x, x^3+x^2+1)],
    '1.2.a': [(P1(1), x^3)],
    '1.2.b': [(x, x^3+1)],
    '1.2.c': [(P1(1), x^3+x)]
}

In [8]:
curves_by_poly[2] = {
    '2.2.ab_a': [(x,x^5+x^2+x)],
    '2.2.ab_c': [(x, x^5+x^3+x)],
    '2.2.ab_d': [(x^3+x+1, x^5+x^4+1)],
    '2.2.ac_e': [(P1(1), x^5+x^4+1)],
    '2.2.a_ab': [(x^2+x, x^5+x^3+x^2+x)],
    '2.2.a_a': [(P1(1), x^5)],
    '2.2.a_b': [(x^2+x+1, x^5+x^3+x^2+1)],
    '2.2.a_c': [(P1(1), x^5+x^4+x^3)],
    '2.2.b_a': [(x, x^5+x)],
    '2.2.b_b': [(x^3+x+1, x^6+x^3+x^2+x)],
    '2.2.b_c': [(x, x^5+x^3+x^2+x)],
    '2.2.b_d': [(x^3+x+1, x^6+x^5+x^4+x^2)],
    '2.2.c_c': [(P1(1), x^5+x^3)],
    '2.2.c_d': [(x^2+x+1, x^5+x^4+x^2+x)], 
    '2.2.c_e': [(P1(1),x^5+x^4)],
    '2.2.d_f': [(x^3+x+1,x^5+x^4+x^3+x)],
}

In [9]:
P3.<x0,x1,x2> = GF(2)[]
curves_by_poly[3] = {
    '3.2.ad_g_ai': [[(x^4+x^2+1,x^8+x+1)], []],
    '3.2.ac_c_ab': [[(x^4+x^2+x+1, x^8+x^7+x^5+x^3+x^2+1)],
                    [x0^4+x0^3*x1+x0^2*x2^2+x0*x2^3+x1^4+x1^3*x2+x1^2*x2^2]],
    '3.2.ac_e_af': [[(x^4+x^2+x,x^8+x^7+x^5+x^4+x)], []],
    '3.2.ab_a_a': [[(x, x^7+x^6+x), (x^4+x^2,x^8+x^5+x^3+x)],
                   [x0^4+x0^3*x1+x0^2*x1*x2+x0*x2^3+x1^4]],
    '3.2.ab_a_c': [[(x, x^7+x^6+x^5+x^2+x)], 
                   [x0^3*x1+x0^2*x1^2+x0^2*x1*x2+x0*x2^3+x1^4,
                    x0^3*x1+x0^2*x1^2+x0*x1^2*x2+x0*x2^3+x1^4]],
    '3.2.ab_a_e': [[], 
                   [x0^4+x0^3*x1+x0^3*x2+x0^2*x1*x2+x0*x1^3+x0*x1^2*x2+x0*x1*x2^2+x0*x2^3+x1^2*x2^2]],
    '3.2.ab_c_ac': [[(x, x^7+x^5+x), (x^4+x^2+1, x^5+x+1)],
                    [x0^4+x0^2*x1^2+x0^2*x1*x2+x0*x2^3+x1^4]],
    '3.2.ab_c_ab': [[], 
                    [x0^4+x0^3*x1+x0^2*x1*x2+x0*x2^3+x1^4+x1^3*x2+x1^2*x2^2]],
    '3.2.ab_c_a': [[(x^4+x^2,x^8+x)], []],
    '3.2.a_a_ab': [[(x^3+x+1, x^7+x^5+x^4+x^3+x+1)], []],
    '3.2.a_a_b': [[(x^3+x+1, x^7+x^6+x^5+x^4+x^3+x^2+x)],
                  [x0^4+x0^3*x2+x0^2*x1^2+x0^2*x1*x2+x0*x1*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.a_a_d': [[], [x0^4+x0^3*x1+x0^3*x2+x0^2*x1*x2+x0*x1^3+x0*x1^2*x2+x0*x2^3+x1^4,
                       x0^3*x2+x0^2*x1^2+x0^2*x1*x2+x0^2*x2^2+x0*x1*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.a_a_f': [[], [x0^3*x1+x0*x2^3+x1^3*x2]],
    '3.2.a_b_a': [[(x^2+x, x^7+x^5+x^2+x), (x^2+x+1, x^7+x^5+x^3+x^2+x+1)],
                  [x0^4+x0*x1^3+x0*x2^3+x1^4]],
    '3.2.a_b_d': [[], [x0^4+x0^3*x1+x0^3*x2+x0^2*x1^2+x0*x2^3+x1^3*x2, 
                       x0^4+x0^3*x1+x0^3*x2+x0^2*x1^2+x0*x2^3+x1^3*x2+x1^2*x2^2]],
    '3.2.a_c_ab': [[(x^4+x^2+x+1, x^8+x^7+x^4+x^2),
                    (x^4+x^2+x+1, x^6+x^5+x^2+1)],
                   [x0^4+x0^2*x1*x2+x0*x1^3+x0*x1^2*x2+x0*x2^3+x1^4]],
    '3.2.a_c_b': [[(x^4+x^2+x+1, x^7+1), (x^4+x^2+x+1, x^8+x^6+x^5+x^4)],
                  [x0^4+x0^3*x2+x0^2*x1*x2+x0*x1^3+x0*x1^2*x2+x0*x2^3+x1^2*x2^2]],
    '3.2.b_b_b': [[(x^4+x+1, x^8+x^7+x^5+x^3)],
                    [x0^3*x2+x0^2*x1*x2+x0*x1^3+x0*x2^3+x1^4]],
    '3.2.b_b_e': [[], [x0^3*x2+x0^2*x1^2+x0^2*x1*x2+x0*x2^3+x1^3*x2]],
    '3.2.b_c_a': [[(x^4+x^2,x^4+x)], []],
    '3.2.b_c_b': [[], [x0^3*x1+x0^2*x1*x2+x0*x2^3+x1^3*x2]],
    '3.2.b_c_c': [[(x,x^7+x^5+x^2+x), (x^4+x^2+1, x^8+x^5+x^4+x)],
                  [x0^4+x0^3*x1+x0^2*x1^2+x0*x1^2*x2+x0*x2^3+x1^4]],
    '3.2.b_c_d': [[], [x0^4+x0^3*x1+x0^2*x1^2+x0*x1*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.b_c_e': [[(x, x^7+x), (x^4+x^2, x^2+x), (x^4+x^2+1, x^8+x^4+x^2+x)],
                  [x0^4+x0^2*x1*x2+x0*x2^3+x1^4]],
    '3.2.b_d_c': [[], [x0^4+x0^3*x2+x0^2*x1^2+x0^2*x1*x2+x0^2*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.b_d_e': [[], [x0^4+x0^3*x2+x0*x1^3+x0*x1^2*x2+x0*x2^3+x1^2*x2^2]],
    '3.2.b_e_d': [[], [x0^4+x0^3*x1+x0^2*x1^2+x0^2*x1*x2+x0*x2^3+x1^3*x2]],
    '3.2.b_e_e': [[], [x0^4+x0^2*x1^2+x0*x1*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.c_d_d': [[], [x0^3*x1+x0^3*x2+x0*x2^3+x1^3*x2]],
    '3.2.c_d_f': [[], [x0^3*x1+x0^3*x2+x0^2*x1^2+x0^2*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.c_e_e': [[], [x0^3*x1+x0^3*x2+x0*x2^3+x1^4]],
    '3.2.c_e_f': [[(x^4+x^2+x, x^7+x^5+x^2+x)],
                    [x0^3*x1+x0^2*x1^2+x0^2*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.c_e_g': [[(P1(1), x^7+x^4)], [x0^4+x0^2*x1^2+x0*x2^3+x1^3*x2]],
    '3.2.c_e_h': [[(x^4+x^2+x+1, x^5+x^3+x^2+x)], []],
    '3.2.d_g_i': [[(x^4+x^2+1, x^4+x)], []],
    '3.2.d_g_k': [[(x^4+x^2+1, x^5+x^2)], []],
    '3.2.d_h_l': [[(x^4+x+1,x^6+x^5+x^3+x)], []],
}

In [10]:
P4.<x0,x1,x2,x3> = GF(2)[]
curves_by_poly[4] = {
    '4.2.a_c_ab_c': [[(x^3+x+1, x^9+x^7+x^2)],
                     [(x0*x1+x3^2, x0^3+x1^3+x0*x1*x2+x2^3+x0*x1*x3+x0*x2*x3),
                      (x0*x1+x3^2, x0^3+x0^2*x1+x1^3+x0*x1*x2+x2^3+x0*x1*x3+x0*x2*x3)]],
    '4.2.a_c_ab_e': [[(x^3+x+1, x^9+x^6+x^2)], 
                     [(x0^2+x0*x1+x1^2+x2*x3, x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.a_c_ab_g': [[(x^3+x+1, x^9+x^7+x+1)],
                     [(x0*x1+x3^2, x1^3+x0*x1*x2+x2^3+x0^2*x3+x0*x1*x3+x0*x2*x3),
                      (x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x1^3+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.a_c_b_c': [[(x^3+x+1, x^9+x^7+x^6+1)],
                    [(x0*x1+x2*x3, x0^3+x1^3+x0^2*x2+x1^2*x2+x0*x2^2+x1*x2^2+x0*x3^2),
                     (x0*x1+x2*x3, x0*x1^2+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x2^3+x0^2*x3+x0*x3^2),
                     (x0*x1+x3^2, x0^3+x1^3+x0*x1*x2+x1^2*x2+x2^3+x0*x2*x3),
                     (x0^2+x0*x1+x1^2+x2*x3, x0^2*x1+x0*x1^2+x1*x2^2+x0^2*x3+x0*x3^2)]],
    '4.2.a_c_b_e': [[(x^3+x+1, x^9+1)],
                    [(x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x0*x1^2+x1^2*x2+x0*x2^2+x0^2*x3+x0*x3^2),
                     (x0^2+x0*x1+x1^2+x2*x3, x0^2*x1+x0*x1^2+x0^2*x2+x1*x2^2+x0^2*x3+x0*x3^2)]],
    '4.2.a_c_b_g': [[(x^3+x+1, x^9+x^7+x^6+x^2+x)],
                    [(x0^2+x0*x1+x1^2+x2*x3, x1^3+x0^2*x2+x0*x2^2+x0*x3^2)]],
    '4.2.a_c_d_a': [[], [(x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x0*x1^2+x1*x2^2+x0^2*x3+x0*x3^2)]],
    '4.2.a_c_d_c': [[], [(x0^2+x0*x1+x1^2+x2*x3, x0^3+x1^3+x0^2*x2+x0*x1*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.a_d_ab_f': [[], [(x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x0^2*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.a_d_ab_h': [[], [(x0*x1+x3^2, x0^3+x0^2*x1+x0*x1^2+x1^3+x0*x1*x2+x2^3+x1^2*x3+x0*x2*x3)]],
    '4.2.a_d_b_f': [[],
                    [(x0*x1+x2*x3, x0^3+x0*x1^2+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x1*x2^2+x0*x3^2),
                     (x0*x1+x3^2, x0^3+x0*x1*x2+x2^3+x0^2*x3+x1^2*x3+x0*x2*x3),
                     (x0*x1+x3^2, x0^3+x1^3+x0*x1*x2+x2^3+x1^2*x3+x0*x2*x3),
                     (x0*x1+x3^2, x0^3+x0^2*x1+x1^3+x0*x1*x2+x2^3+x1^2*x3+x0*x2*x3),
                     (x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x0*x2^2+x0*x3^2)]],
    '4.2.a_d_b_h': [[], [(x0*x1+x2*x3, x0^3+x0*x1^2+x1^3+x0^2*x2+x1^2*x2+x0*x2^2+x0^2*x3+x0*x1*x3+x0*x3^2),
                          (x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2),
                          (x0^2+x0*x1+x1^2+x2*x3, x0^3+x1^3+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.a_d_d_f': [[], [(x0*x1+x3^2, x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x2^3+x1^2*x3+x0*x2*x3)]],
    '4.2.a_e_b_k': [[], [(x0*x1+x3^2, x0^3+x0^2*x1+x1^3+x0*x1*x2+x2^3+x0^2*x3+x0*x1*x3+x0*x2*x3)]],
    '4.2.b_b_a_b': [[], [(x0*x1+x3^2, x0^2*x1+x1^2*x2+x2^3+x0^2*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_b_a_d': [[], [(x0*x1+x2*x3, x0^3+x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.b_b_c_d': [[], [(x0*x1+x2*x3, x0*x1^2+x1^3+x0^2*x2+x0*x1*x2+x0*x2^2+x0*x1*x3+x0*x3^2),
                         (x0*x1+x3^2, x0^2*x2+x1^2*x2+x2^3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_b_c_f': [[], [(x0*x1+x2*x3, x0^2*x1+x0*x1^2+x1^3+x0^2*x2+x0*x1*x2+x0*x2^2+x0*x3^2),
                         (x0*x1+x3^2, x0^3+x0^2*x1+x0^2*x2+x1^2*x2+x2^3+x0^2*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_b_c_h': [[], [(x0*x1+x3^2, x0^3+x1^2*x2+x2^3+x1^2*x3+x0*x2*x3),
                         (x0^2+x0*x1+x1^2+x2*x3, x1^3+x0*x1*x2+x0*x2^2+x0*x3^2)]],
    '4.2.b_b_a_b': [[], [(x0*x1+x3^2, x0^2*x1+x1^2*x2+x2^3+x0^2*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_b_a_d': [[], [(x0*x1+x2*x3, x0^3+x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.b_c_a_a': [[(x^2, x^9+x^7+x+1)], []],
    '4.2.b_c_a_c': [[(x, x^9+x^7+x^5+x^3+x^2+1)], []],
    '4.2.b_c_a_e': [[], [(x0^2+x0*x1+x1^2+x2*x3, x0*x1*x2+x1^2*x2+x0*x2^2+x0*x3^2)]],
    '4.2.b_c_c_c': [[(x, x^9+x^7+x^3+1)], []],
    '4.2.b_c_c_e': [[(x^2, x^9+x^5+x^4+x), (x^2, x^9+x^4+x+1)], []],
    '4.2.b_c_c_g': [[(x, x^9+1)],
                    [(x0*x1+x2*x3, x0^3+x0*x1^2+x0^2*x2+x1^2*x2+x0*x2^2+x1^2*x3+x0*x3^2),
                     (x0*x1+x2*x3, x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x1*x2^2+x0^2*x3+x0*x3^2),
                     (x0*x1+x3^2, x1^3+x0^2*x2+x0*x1*x2+x2^3+x0^2*x3),
                     (x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.b_b_c_h': [[], [(x0*x1+x3^2, x0^3+x1^2*x2+x2^3+x1^2*x3+x0*x2*x3),
                         (x0^2+x0*x1+x1^2+x2*x3, x1^3+x0*x1*x2+x0*x2^2+x0*x3^2)]],
    '4.2.b_c_d_h': [[(x^4+x^2+x, x^9+x^8+x^2+1)], 
                    [(x0*x1+x2*x3, x0^3+x0*x1^2+x1^3+x0*x1*x2+x0*x2^2+x0*x1*x3+x0*x3^2),
                     (x0^2+x0*x1+x1^2+x2*x3, x0^2*x1+x1^3+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.b_c_d_j': [[], [(x0*x1+x3^2, x0^3+x0^2*x1+x2^3+x0^2*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_c_e_e': [[(x^2, x^9+x^7+x^5+x)], []],
    '4.2.b_d_c_h': [[], [(x0*x1+x2*x3, x0^3+x0^2*x1+x0*x1^2+x0*x1*x2+x1^2*x2+x0*x2^2+x0^2*x3+x1^2*x3+x0*x3^2),
                         (x0*x1+x2*x3, x0^3+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x0^2*x3+x1^2*x3+x0*x3^2),
                         (x0*x1+x3^2, x0^3+x0^2*x1+x0*x1^2+x1^3+x2^3+x0*x1*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_d_e_j': [[], [(x0*x1+x3^2, x0^3+x1^3+x2^3+x0*x1*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_e_c_i': [[], [(x0*x1+x3^2, x0^3+x1^3+x2^3+x0*x2*x3)]],
    '4.2.b_e_e_k': [[], [(x0*x1+x3^2, x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x2^3+x0^2*x3),
                         (x0^2+x0*x1+x1^2+x2*x3, x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.c_e_h_k': [[(x^3+x+1, x^9+x^7)], []],
    '4.2.c_g_i_q': [[], 
                    [(x0*x1+x2*x3,x0^3+x0*x1^2+x0^2*x2+x1^2*x2+x0*x2^2+x1*x2^2+x0^2*x3+x0*x1*x3+x1^2*x3+x0*x3^2),
                     (x0*x1+x2*x3,x1^3+x0*x1*x2+x1^2*x2+x0*x2^2+x1*x2^2+x0^2*x3+x0*x1*x3+x1^2*x3+x0*x3^2)]],
    '4.2.d_f_i_n': [[], [(x0*x1+x2*x3, x0^2*x1+x0*x1^2+x0^2*x2+x0*x1*x2+x1^2*x2+x1*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]],
    '4.2.d_f_k_s': [[], 
                    [(x0*x1+x3^2, x0^2*x1+x0^2*x2+x0*x1*x2+x1^2*x2+x2^3+x0^2*x3+x0*x1*x3+x1^2*x3)]],
    '4.2.d_h_l_r': [[(x^5+x^2+1, x^4+x^2)], 
                    [(x0*x1+x2*x3,x0^2*x1+x0*x1^2+x1^3+x0*x1*x2+x1^2*x2+x0*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]],
    '4.2.d_i_o_x': [[],
                    [(x0*x1+x2*x3,x0^2*x2+x0*x1*x2+x0*x2^2+x1*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]],
    '4.2.e_j_q_z': [[], [(x0*x1+x2*x3, x0^2*x1+x1^3+x0*x1*x2+x1^2*x2+x0*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]],
    '4.2.e_k_u_bg': [[], [(x0*x1+x2*x3, x0^2*x2+x1^2*x2+x1*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]],
    '4.2.f_o_bc_bs': [[], [(x0*x1+x2*x3, x0*x1^2+x1^3+x0^2*x2+x1^2*x2+x0*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]]
}

Populate the table in genus 5 using data from Dragutinović.

In [11]:
with open("genus5byweilpoly.txt", "r") as f:
    s = f.read()
P.<T> = QQ[]
tmp0.<xtmp> = GF(2)[]
tmp1.<x0tmp,x1tmp,x2tmp,x3tmp,x4tmp> = GF(2)[]
tmp = sage_eval(s, locals={'T': P.gen(), 'x':tmp0.gen(), 'x0': tmp1.gens()[0],'x1': tmp1.gens()[1], 
                           'x2': tmp1.gens()[2], 'x3': tmp1.gens()[3],  'x4': tmp1.gens()[4]})

In [12]:
P5.<x0,x1,x2,x3,x4> = GF(2)[]
curves_by_poly[5] = {}
for u in tmp:
    s = label_from_weil_poly(u)
    curves_by_poly[5][s] = [[(P1(a), P1(b)) for (a,b) in tmp[u][0]],
                            [f(P3.gens()[0], P3.gens()[1], P3.gens()[2], 0, 0) for f in tmp[u][1]],
                            [tuple(gen(*P5.gens()) for gen in gens) for gens in tmp[u][2]]]

Manually construct curves of genus 6 and 7 that will give rise to extensions with relative class number 1.

In [13]:
Pxy.<x,y> = GF(2)[]
curves_by_poly[6] = {}
curves_by_poly[6]['6.2.c_h_k_z_bd_cg'] = [[], [x^2*y^5 + (x^3 + x)*y^4 + x^4*y^3 + 
                                               (x^5 + x^4 + x^3 + x^2 + x + 1)*y^2 + 
                                               (x^6 + x^3 + x^2)*y + x^7 + x^3 + x + 1]]
curves_by_poly[6]['6.2.d_j_t_bn_cl_du'] = [[], [(x^2 + x + 1)*y^4 + (x^3 + x + 1)*y^3 + (x^4 + x^2 + 1)*y^2 + 
                                                (x^5 + x^4 + 1)*y + x^5 + x^4 + x^3 + x]]

In [14]:
u = weil_poly_from_point_count((6,18,12,18,6,60,174), 7)
s = label_from_weil_poly(u)
s

'7.2.d_l_z_cf_dx_gs_js'

In [15]:
curves_by_poly[7] = {}
curves_by_poly[7][s] = [[], [y^4 + (x^6+x^4+x^3+x^2+1)*y^2 + (x^6+x^4+x^3+x^2)*y + x^10 + x^9 + x^7 + x^6]]

# Magma function fields

Produce Magma function fields of curves with a given zeta function.

In [16]:
def function_fields_by_weil_poly(s, m=1):
    if m == 1:
        F0 = magma.FunctionField(GF(2))
    else:
        K.<a> = GF(2^m)
        F0 = magma.FunctionField(K)
    F0.AssignNames('x')
    S = magma.PolynomialRing(F0)
    S.AssignNames('y')
    l = s.split(".")
    g = Integer(l[0])
    if s in curves_by_poly[g]:
        ans = []
        # Hyperelliptics.
        m = curves_by_poly[g][s] if g <= 2 else curves_by_poly[g][s][0]
        for (u,v) in m:
            yield magma.HyperellipticCurve(v,u).FunctionField().AlgorithmicFunctionField()
        # Plane quartics for g=3.
        if g == 3:
            proj = magma.ProjectiveSpace(P3)
            for u in curves_by_poly[g][s][1]:
                yield proj.Scheme([u]).FunctionField().AlgorithmicFunctionField()
        # Genus 4 curves.
        if g == 4:
            proj = magma.ProjectiveSpace(P4)
            for (u,v) in curves_by_poly[g][s][1]:
                yield proj.Scheme([u,v]).FunctionField().AlgorithmicFunctionField()
        # Genus 5 curves.
        if g == 5:
            proj = magma.ProjectiveSpace(P3)
            for u in curves_by_poly[g][s][1]:
                yield proj.Scheme([u]).FunctionField().AlgorithmicFunctionField()        
            proj = magma.ProjectiveSpace(P5)        
            for (u,v,w) in curves_by_poly[g][s][2]:
                yield proj.Scheme([u,v,w]).FunctionField().AlgorithmicFunctionField()
        # Genus 6 and 7 curves, represented as singular plane curves.
        if g == 6 or g == 7:
            proj = magma.AffineSpace(Pxy)
            for u in curves_by_poly[g][s][1]:
                yield proj.Scheme(u).FunctionField().AlgorithmicFunctionField()

Check that the zeta functions of these curves are as indicated.

In [17]:
P.<T> = QQ[]
for g in range(1, 8):
    for s in curves_by_poly[g]:
        for F in function_fields_by_weil_poly(s):
            u = F.ZetaFunction().Numerator().sage()(T).reverse()
            assert label_from_weil_poly(u) == s

# Construction of extensions

Construct extensions with relative class number 1.

In [18]:
covers_by_poly = {}

In [19]:
def match_weil_poly_general(s, d, delta=0, verbose=False):
    l = s.split(".")
    g = Integer(l[0])
    g1 = g + (d-1)*(g-1) + delta
    ans = []
    for F in function_fields_by_weil_poly(s):
        ans.append((s, match_weil_poly(F, d, delta=delta, verbose=verbose)))
    if ans:
        if (d, g, g1) not in covers_by_poly:
            covers_by_poly[d, g, g1] = {}
        if s not in covers_by_poly[d, g, g1]:
            covers_by_poly[d, g, g1][s] = {}
        for (_, l) in ans:
            for (F, F1) in l:
                if F not in covers_by_poly[d, g, g1][s]:
                    covers_by_poly[d, g, g1][s][F] = []
                if F1 not in covers_by_poly[d, g, g1][s][F]:
                    covers_by_poly[d, g, g1][s][F].append(F1)
    return ans

In [20]:
for (d, g, g1) in candidates:
    print(d, g, g1)
    delta = g1-g-(d-1)*(g-1)
    covers_by_poly[d, g, g1] = {}
    for s in candidates[d,g,g1]:
        match_weil_poly_general(s, d, delta=g1-g-(d-1)*(g-1))

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 2 6
4 3 9
5 2 6
6 2 7
7 2 8


Verify that no base field is repeated.

In [21]:
for (d, g, g1) in covers_by_poly:
    print(d,g,g1)
    for s in covers_by_poly[d,g,g1]:
        if covers_by_poly[d,g,g1][s]:
            l1 = list(covers_by_poly[d,g,g1][s].keys())
            l2 = isomorphism_class_reps(l1)
            assert len(l1) == len(l2)

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 2 6
4 3 9
5 2 6
6 2 7
7 2 8


Verify that each base curve gives rise to a unique cover of the specified degree and genus.

In [23]:
for (d, g, g1) in covers_by_poly:
    print(d,g,g1)
    for s in covers_by_poly[d,g,g1]:
        for F, l1 in covers_by_poly[d,g,g1][s].items():
            if len(l1) > 1:
                assert len(l1) == 2
                F1 = l1[0].RationalExtensionRepresentation()
                F2 = l1[1].RationalExtensionRepresentation()
                assert F1.IsIsomorphic(F2)

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 2 6
4 3 9
5 2 6
6 2 7
7 2 8


Count cyclic covers arising from this calculation.

In [36]:
sum(1 for (d,g,g1) in covers_by_poly for s in covers_by_poly[d,g,g1] for l1 in covers_by_poly[d,g,g1][s].values() if l1)

64

# Populating the spreadsheet

Convert the covers into numerical data in preparation for writing out to the spreadsheet.

In [24]:
l = []
for (d, g, g1) in candidates:
    print(d, g, g1)
    if (d, g, g1) in covers_by_poly:
        for s in covers_by_poly[d, g, g1]:
            for F in covers_by_poly[d, g, g1][s]:
                for F1 in covers_by_poly[d, g, g1][s][F]:
                    u1 = T.parent()(magma_poly_list(F.ZetaFunction().Numerator())).reverse()
                    v1 = T.parent()(magma_poly_list(F1.ZetaFunction().Numerator())).reverse()
                    l.append((d, g, g1, point_count_from_weil_poly(u1, 13),
                              point_count_from_weil_poly(v1, 13)))

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 2 6
4 3 9
5 2 6
6 2 7
7 2 8


Write the results back to the spreadsheet. In the process, we check that every covering we found gives a pair of Weil polynomials from our original list.

In [25]:
df['Cyclic'] = 'No'
used_pols = []
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    ct1 = eval(r["Counts of C"])
    ct2 = eval(r["Counts of C'"])
    if (d, g, g1, ct1, ct2) in l:
        df.loc[i, "Cyclic"] = "Yes"
        used_pols.append((d, g, g1, ct1, ct2))
assert all(t in used_pols for t in l)

In [26]:
df.to_excel('../Shared/polys.xlsx', sheet_name='Weil polynomials', merge_cells=True, freeze_panes=(int(1),int(1)))

# Tables

Generate LaTeX-formatted tables of the data for $d=2$ and $d>2$.

In [27]:
R0.<x> = GF(2)[]
R1.<y> = R0[]
print(r'\begin{tabular}{c|c|c|c|c}')
print(r"$d$ & $g_F$ & $g_{F'}$ & $J(C)$ & $F$ \\")
for (d, g, g1) in candidates:
    if d > 2:
        if any(covers_by_poly[d, g, g1][s] for s in covers_by_poly[d, g, g1]):
            print(r'\hline')
            for s in covers_by_poly[d, g, g1]:
                if covers_by_poly[d,g,g1][s]:
                    for F in covers_by_poly[d, g, g1][s]:
                        F1 = F.RationalExtensionRepresentation()
                        try:
                            u = F1.DefiningPolynomial().sage()
                        except SyntaxError:
                            u = magma_poly_list(F.DefiningPolynomial())
                            u = [R0(magma_poly_list(i.Numerator())) / R0(magma_poly_list(i.Denominator())) for i in u]
                            u = R1(u)
                        print(r'${}$ & ${}$ & ${}$ & $\avlink{{{}}}$ & ${}$ \\'.format(d, g, g1,
                                                        re.sub('_', r'\_', s),
                                                        latex(u)))
print(r'\end{tabular}')

\begin{tabular}{c|c|c|c|c}
$d$ & $g_F$ & $g_{F'}$ & $J(C)$ & $F$ \\
\hline
$3$ & $2$ & $4$ & $\avlink{2.2.ac\_e}$ & $y^{2} + y + x^{5} + x^{4} + 1$ \\
$3$ & $2$ & $4$ & $\avlink{2.2.b\_b}$ & $y^{2} + \left(x^{3} + x + 1\right) y + x^{6} + x^{3} + x^{2} + x$ \\
\hline
$3$ & $2$ & $6$ & $\avlink{2.2.a\_c}$ & $y^{2} + y + x^{5} + x^{4} + x^{3}$ \\
$3$ & $2$ & $6$ & $\avlink{2.2.b\_c}$ & $y^{2} + x y + x^{5} + x^{3} + x^{2} + x$ \\
$3$ & $2$ & $6$ & $\avlink{2.2.b\_d}$ & $y^{2} + \left(x^{3} + x + 1\right) y + x^{6} + x^{5} + x^{4} + x^{2}$ \\
\hline
$3$ & $3$ & $7$ & $\avlink{3.2.a\_b\_a}$ & $y^{4} + \left(x^{3} + 1\right) y + x^{4}$ \\
$3$ & $3$ & $7$ & $\avlink{3.2.a\_b\_d}$ & $y^{3} + x^{2} y^{2} + x^{3} y + x^{4} + x^{3} + x$ \\
$3$ & $3$ & $7$ & $\avlink{3.2.b\_b\_b}$ & $y^{3} + x y^{2} + \left(x^{3} + 1\right) y + x^{4}$ \\
$3$ & $3$ & $7$ & $\avlink{3.2.b\_b\_e}$ & $y^{3} + \left(x^{2} + x\right) y^{2} + y + x^{3}$ \\
$3$ & $3$ & $7$ & $\avlink{3.2.b\_c\_b}$ & $x y^{3} + x y^{2} + 

In [28]:
R0.<x> = GF(2)[]
R1.<y> = R0[]
print(r'\begin{tabular}{c|c|c|p{7.5cm}}')
print(r"$g_F$ & $g_{F'}$ & $J(C)$ & $F$ \\")
for (d, g, g1) in candidates:
    if d == 2:
        if covers_by_poly[d, g, g1]:
            print(r'\hline')
            if g == 6:
                print(r'\hline')
            for s in covers_by_poly[d, g, g1]:
                if covers_by_poly[d,g,g1][s]:
                    for F in covers_by_poly[d, g, g1][s]:
                        F1 = F.RationalExtensionRepresentation()
                        try:
                            u = F1.DefiningPolynomial().sage()
                        except SyntaxError:
                            u = magma_poly_list(F1.DefiningPolynomial())
                            u = [R0(magma_poly_list(i.Numerator())) / R0(magma_poly_list(i.Denominator())) for i in u]
                            u = R1(u)
                        if g < 7: 
                            s1 = r"\avlink{{{}}}".format(s)
                        else:
                            s1 = str(tuple(Integer(F.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, g+1)))
                        print(r'${}$ & ${}$ & ${}$ & ${}$ \\'.format(g, g1,
                                                        re.sub('_', r'\_', s1),
                                                        latex(u)))
print(r'\end{tabular}')

\begin{tabular}{c|c|c|p{7.5cm}}
$g_F$ & $g_{F'}$ & $J(C)$ & $F$ \\
\hline
$2$ & $3$ & $\avlink{2.2.ab\_c}$ & $y^{2} + x y + x^{5} + x^{3} + x$ \\
$2$ & $3$ & $\avlink{2.2.b\_c}$ & $y^{2} + x y + x^{5} + x^{3} + x^{2} + x$ \\
\hline
$2$ & $4$ & $\avlink{2.2.a\_a}$ & $y^{2} + y + x^{5}$ \\
$2$ & $4$ & $\avlink{2.2.a\_c}$ & $y^{2} + y + x^{5} + x^{4} + x^{3}$ \\
$2$ & $4$ & $\avlink{2.2.b\_b}$ & $y^{2} + \left(x^{3} + x + 1\right) y + x^{6} + x^{3} + x^{2} + x$ \\
\hline
$2$ & $5$ & $\avlink{2.2.b\_d}$ & $y^{2} + \left(x^{3} + x + 1\right) y + x^{6} + x^{5} + x^{4} + x^{2}$ \\
$2$ & $5$ & $\avlink{2.2.c\_e}$ & $y^{2} + y + x^{5} + x^{4}$ \\
\hline
$3$ & $5$ & $\avlink{3.2.ad\_g\_ai}$ & $y^{2} + \left(x^{4} + x^{2} + 1\right) y + x^{8} + x + 1$ \\
$3$ & $5$ & $\avlink{3.2.ab\_a\_c}$ & $x y^{3} + \left(x^{2} + x\right) y^{2} + y + x^{4}$ \\
$3$ & $5$ & $\avlink{3.2.ab\_a\_c}$ & $x y^{3} + x^{2} y^{2} + \left(x^{2} + 1\right) y + x^{4}$ \\
$3$ & $5$ & $\avlink{3.2.ab\_c\_ac}$ & $y^{2} + x y 